# nb16 — The Repulsive Void: Pe < 0 Regime

**First THRML exploration of negative-Pe substrates — the repulsive void.**

All previous substrates (crypto, gambling, AI) had Pe > 0: drift toward high engagement.
EXP-022 found the first Pe < 0 case: Jehovah's Witnesses, retention 37%, Pe = −8.92.
Buddhist sat exactly at Pe = 0 (50% retention, maximum entropy).

This notebook asks: what does the THRML Ising model look like in the Pe < 0 regime?
Is it a boundary case or a full new physics?

$$
\text{Pe}(c) = K \cdot \sinh\!\bigl(2(b_\alpha - c\,b_\gamma)\bigr)
$$

Pe < 0 requires $b_\text{net} < 0$, i.e., $c > b_\alpha / b_\gamma$. This is already
in the formula — no new mechanism needed. The sign flips when constraint exceeds drive.

**Three questions:**
1. At what $c$ does Pe = 0? Is it K-dependent?
2. What do THRML trajectories look like when Pe < 0 (drift toward EXIT)?
3. Where does JW sit in the extended phase diagram?

**Relates to:** EXP-022 (Pew retention data), nb10 (cross-domain calibration),
nb12 (K-scaling), Papers 4C, 4D.

In [ ]:
import jax
import jax.numpy as jnp
import jax.random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import numpy as np
from scipy.special import expit

from thrml.block_management import Block
from thrml.block_sampling import sample_states, SamplingSchedule
from thrml.models.ising import IsingEBM, IsingSamplingProgram
from thrml.pgm import SpinNode

# Canonical THRML parameters (EXP-001, never refit)
b_alpha = 0.5 * np.log(0.85 / 0.15)              # UU equilibrium theta*=0.85
b_gamma = b_alpha - 0.5 * np.log(0.06 / 0.94)    # GG equilibrium theta*=0.06
K = 16

def pe_analytic(c, b_a=b_alpha, b_g=b_gamma, k=K):
    return k * np.sinh(2.0 * (b_a - c * b_g))

def c_from_pe_signed(pe_val, b_a=b_alpha, b_g=b_gamma, k=K):
    """Invert Pe = K*sinh(2*b_net), signed. Handles Pe < 0."""
    b_net = np.arcsinh(pe_val / k) / 2.0  # signed — b_net < 0 when Pe < 0
    return (b_a - b_net) / b_g

def retention_to_pe(theta, k=K):
    """Convert retention rate to Pe via half-logit. Direct from EXP-022."""
    b_net = 0.5 * np.log(theta / (1.0 - theta))
    return k * np.sinh(2.0 * b_net)

def c_crit(k_val=K, b_a=b_alpha, b_g=b_gamma):
    return (b_a - np.arcsinh(1.0 / k_val) / 2.0) / b_g

# The Pe=0 boundary — K-INDEPENDENT
c_zero = b_alpha / b_gamma  # where b_net = 0, Pe = 0 for ANY K
c_boundary = c_crit()

print(f'b_alpha = {b_alpha:.4f},  b_gamma = {b_gamma:.4f},  K = {K}')
print(f'c_crit(K=16)  = {c_boundary:.4f}  (drift/diffusion boundary, K-dependent)')
print(f'c_zero        = {c_zero:.4f}  (Pe=0, b_net=0, K-INDEPENDENT)')
print(f'diffusion gap = {c_zero - c_boundary:.4f}  (c_zero - c_crit at K=16)')
print()
# Verify JW
jw_theta = 0.37
jw_pe = retention_to_pe(jw_theta)
jw_c = float(c_from_pe_signed(jw_pe))
jw_b_net = b_alpha - jw_c * b_gamma
print(f'JW: theta*={jw_theta}, Pe={jw_pe:.3f}, c={jw_c:.4f}, b_net={jw_b_net:.4f}')
print(f'  c_JW > c_zero: {jw_c:.4f} > {c_zero:.4f} =', jw_c > c_zero, '(repulsive void confirmed)')

## Three Void Regimes

Pe = K * sinh(2 * b_net) where b_net = b_alpha - c * b_gamma.

The sign of Pe is entirely determined by the sign of b_net:

| Regime | Condition | Pe sign | Equilibrium | Example |
|--------|-----------|---------|-------------|---------|
| **Attractive void** | c < c_crit | Pe > 1 | theta* > 0.5, drift toward high engagement | Crypto, Gambling |
| **Diffusion zone** | c_crit <= c < c_zero | 0 < Pe < 1 | theta* in (0.5, drift onset) | AI-GG |
| **Null void** | c = c_zero | Pe = 0 | theta* = 0.5, maximum entropy | Buddhist |
| **Repulsive void** | c > c_zero | Pe < 0 | theta* < 0.5, drift toward EXIT | JW, Mainline |

**Key result:** c_zero = b_alpha / b_gamma is K-INDEPENDENT.
At any hardware scale K, the Pe=0 boundary is at the same constraint level.
Contrast: c_crit(K) is K-dependent and converges to c_zero as K -> infinity.

In the K->infinity limit, the diffusion zone vanishes entirely:
every substrate is either attracting (c < c_zero) or repelling (c > c_zero).
The null void is a knife-edge in the thermodynamic limit.

In [ ]:
# Figure 1: Extended Pe(c) curve — all three regimes
c_range = np.linspace(0.0, 0.60, 600)
pe_curve = pe_analytic(c_range)

fig, ax = plt.subplots(figsize=(11, 6))

# Three zone shading
ax.axvspan(0.0, c_boundary, alpha=0.08, color='#e74c3c', label='Attractive void (Pe > 1)')
ax.axvspan(c_boundary, c_zero, alpha=0.12, color='#f39c12', label='Diffusion zone (0 < Pe < 1)')
ax.axvspan(c_zero, 0.60, alpha=0.08, color='#3498db', label='Repulsive void (Pe < 0)')

# Pe(c) curve
ax.plot(c_range, pe_curve, 'k-', linewidth=2.5, zorder=5,
        label='Pe(c) = K * sinh(2(b_alpha - c*b_gamma))')

# Reference lines
ax.axhline(y=0,  color='black',   linewidth=1.2, zorder=4)
ax.axhline(y=1,  color='#e74c3c', linewidth=1.0, linestyle='--', alpha=0.7)
ax.axhline(y=-1, color='#e74c3c', linewidth=1.0, linestyle='--', alpha=0.7)
ax.axvline(x=c_boundary, color='#e74c3c', linewidth=1.0, linestyle=':', alpha=0.7)
ax.axvline(x=c_zero, color='#2c3e50', linewidth=2.0, linestyle='--', alpha=0.9,
           label=f'c_zero = {c_zero:.4f} (K-invariant, Pe=0 boundary)')

# Empirical points
points = [
    ('AI-UU',    float(c_from_pe_signed(7.94)),          7.94,   '#e74c3c', 'D'),
    ('ETH',      float(c_from_pe_signed(3.74)),           3.74,   '#627eea', '^'),
    ('AI-GG',    float(c_from_pe_signed(0.76)),           0.76,   '#2ecc71', 'D'),
    ('Buddhist', c_zero,                                   0.00,   '#FF6600', 'o'),
    ('Mainline', float(c_from_pe_signed(-3.23)),          -3.23,  '#6699CC', 's'),
    ('JW',       float(c_from_pe_signed(-8.92)),          -8.92,  '#CC0066', 'P'),
]

for name, c_pt, pe_pt, color, marker in points:
    ax.scatter(c_pt, pe_pt, s=180, marker=marker, color=color,
               edgecolors='white', linewidth=1.0, zorder=10)
    va = 'bottom' if pe_pt >= 0 else 'top'
    dy = 1.0 if pe_pt >= 0 else -1.0
    ax.text(c_pt + 0.007, pe_pt + dy, name, fontsize=9, color=color,
            va=va, fontweight='bold')

# Zone labels
ax.text(0.19, 22, 'Attractive void\nPe > 1\ndrift toward stay',
        ha='center', va='center', fontsize=9, color='#c0392b',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
ax.text((c_boundary + c_zero)/2, 8,
        f'Diffusion\nwidth = {c_zero-c_boundary:.3f}',
        ha='center', va='center', fontsize=7.5, color='#d35400',
        bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8))
ax.text(0.49, -6, 'Repulsive void\nPe < 0\ndrift toward exit',
        ha='center', va='center', fontsize=9, color='#2980b9',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

ax.set_xlabel('Constraint level c', fontsize=13)
ax.set_ylabel('Peclet number Pe  (signed)', fontsize=13)
ax.set_xlim(0.0, 0.60)
ax.set_ylim(-15, 45)
ax.set_title(
    f'Pe(c) — three void regimes\n'
    f'K={K}, b_alpha={b_alpha:.3f}, b_gamma={b_gamma:.3f}  (canonical, EXP-001)\n'
    f'c_zero={c_zero:.4f} is K-invariant; c_crit={c_boundary:.4f} is K-dependent',
    fontsize=10,
)
ax.legend(loc='upper right', fontsize=8.5, framealpha=0.9)
ax.grid(True, linestyle=':', alpha=0.3)
plt.tight_layout()
plt.savefig('nb16_extended_pe_curve.svg', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: nb16_extended_pe_curve.svg')

## JW Case: Inferring c from Retention

For any substrate with a known equilibrium theta*, we can infer:
- b_net = 0.5 * log(theta* / (1 - theta*))   [half-logit]
- c = (b_alpha - b_net) / b_gamma             [invert the Pe formula]

For JW (theta* = 0.37): b_net = -0.266, c = 0.505 > c_zero = 0.387.
The constraint term has exceeded the drive term — b_net flipped sign.

This does NOT require a new mechanism in THRML. The same formula covers it.
The interpretation comes from EXP-022: the 'constraint' in JW is not just regulatory
— it is punitive. Shunning creates an anti-responsive R that, above c_zero,
reverses the gradient entirely. The void becomes repulsive.

**All denominations on the c axis:**

In [ ]:
denominations = [
    ('JW',             0.37),
    ('Mainline Prot.', 0.45),
    ('Buddhist',       0.50),
    ('Nones 2015',     0.53),
    ('Catholic',       0.59),
    ('Mormon',         0.64),
    ('Evangelical',    0.65),
    ('Orthodox',       0.73),
    ('Nones 2023',     0.73),
    ('Jewish',         0.75),
    ('Muslim',         0.77),
    ('Hindu',          0.80),
]

print(f'{"Denomination":<22} {"theta*":>6} {"b_net":>8} {"Pe":>8} {"c_implied":>10} {"c-c_zero":>10}  regime')
print('-' * 82)
for name, theta in denominations:
    pe = retention_to_pe(theta)
    b_net = 0.5 * np.log(theta / (1.0 - theta))
    c_imp = float(c_from_pe_signed(pe))
    delta = c_imp - c_zero
    if pe < -1:
        regime = 'repulsive void'
    elif pe < 0:
        regime = 'repulsive (weak)'
    elif pe < 1:
        regime = 'null / diffusion'
    else:
        regime = 'attractive void'
    print(f'{name:<22} {theta:>6.2f} {b_net:>8.4f} {pe:>8.2f} {c_imp:>10.4f} {delta:>+10.4f}  {regime}')

print()
print(f'c_zero  = {c_zero:.4f}  (K-invariant Pe=0 boundary)')
print(f'c_crit  = {c_boundary:.4f}  (K=16 drift/diffusion boundary)')
print()
print(f'Substrates above c_zero (Pe < 0): JW, Mainline, Buddhist=exactly')
print(f'Buddhist theta*=0.50 -> b_net=0.000 -> c={c_from_pe_signed(0.0):.4f} = c_zero exactly')

## THRML Trajectories: Three Regimes

Run the Ising sampler at b_net > 0 (ETH), b_net = 0 (Buddhist), b_net < 0 (JW).
All start from theta = 0.5 (random init) for a fair comparison.

**Expected:**
- ETH (b_net = +0.107): theta drifts UP toward theta* = 0.56 — but THRML transient shows directional drift
- Buddhist (b_net = 0.000): theta random-walks, no preferred direction, stays near 0.5
- JW (b_net = -0.266): theta drifts DOWN toward theta* = 0.37

The sampler handles negative b_net natively — no modifications needed.
The Pe sign is encoded in the drift direction of the theta trajectory.

In [ ]:
def build_ising_chain(b_net, K=16):
    nodes = [SpinNode() for _ in range(K)]
    edges = [(nodes[i], nodes[i+1]) for i in range(K-1)]
    biases = jnp.full(K, float(b_net))
    J = 0.02 / K
    weights = jnp.full(K-1, J)
    beta = jnp.array(1.0)
    model = IsingEBM(nodes, edges, biases, weights, beta)
    return model, nodes

def run_ising_random_init(key, b_net, K=16, N_samples=120):
    """Run K-spin Ising from random theta~0.5 init. Returns (theta_trajectory, Pe_signed)."""
    model, nodes = build_ising_chain(b_net, K)
    even = [nodes[i] for i in range(0, K, 2)]
    odd  = [nodes[i] for i in range(1, K, 2)]
    free_blocks = [Block(even), Block(odd)]
    program = IsingSamplingProgram(model, free_blocks, [])
    schedule = SamplingSchedule(0, N_samples, 1)
    # Random init near theta=0.5
    key, k1, k2, subkey = jax.random.split(key, 4)
    even_init = jax.random.bernoulli(k1, 0.5, shape=(len(even),))
    odd_init  = jax.random.bernoulli(k2, 0.5, shape=(len(odd),))
    init = [even_init, odd_init]
    samples = sample_states(subkey, program, schedule, init, [], [Block(nodes)])
    spins = np.array(samples[0].astype(np.float32))
    theta = spins.mean(axis=-1)
    # Signed Pe
    dm = np.diff(theta)
    v = np.mean(dm)
    D = np.var(dm) / 2.0
    pe_signed = float(v / D) if D > 1e-12 else 0.0
    return theta, pe_signed

# Three cases
b_net_eth      = b_alpha - float(c_from_pe_signed(3.74)) * b_gamma
b_net_buddhist = 0.0
b_net_jw       = b_alpha - jw_c * b_gamma

theta_star_eth      = float(expit(2.0 * b_net_eth))
theta_star_buddhist = 0.5
theta_star_jw       = float(expit(2.0 * b_net_jw))

print(f'ETH      b_net={b_net_eth:.4f}  theta*={theta_star_eth:.3f}  Pe_analytic={pe_analytic(float(c_from_pe_signed(3.74))):.2f}')
print(f'Buddhist b_net={b_net_buddhist:.4f}  theta*={theta_star_buddhist:.3f}  Pe_analytic=0.00')
print(f'JW       b_net={b_net_jw:.4f}  theta*={theta_star_jw:.3f}  Pe_analytic={jw_pe:.2f}')

key = jax.random.key(16)
key, k1 = jax.random.split(key)
theta_eth, pe_eth_t = run_ising_random_init(k1, b_net_eth)
key, k2 = jax.random.split(key)
theta_bud, pe_bud_t = run_ising_random_init(k2, b_net_buddhist)
key, k3 = jax.random.split(key)
theta_jw, pe_jw_t  = run_ising_random_init(k3, b_net_jw)

print()
print(f'THRML Pe (signed, from trajectory):')
print(f'  ETH:      {pe_eth_t:+.3f}')
print(f'  Buddhist: {pe_bud_t:+.3f}')
print(f'  JW:       {pe_jw_t:+.3f}')
print(f'  Signs: ETH>0={pe_eth_t>0}, JW<0={pe_jw_t<0}')

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

cases = [
    (theta_eth,  theta_star_eth,  pe_eth_t,  3.74,  '#627eea', 'Attractive void\nETH  Pe = +3.74'),
    (theta_bud,  theta_star_buddhist, pe_bud_t, 0.00, '#FF6600', 'Null void\nBuddhist  Pe = 0.00'),
    (theta_jw,   theta_star_jw,   pe_jw_t,  -8.92,  '#CC0066', 'Repulsive void\nJW  Pe = -8.92'),
]

for ax, (theta_traj, theta_star, pe_t, pe_emp, color, title) in zip(axes, cases):
    rounds = np.arange(len(theta_traj))
    ax.plot(rounds, theta_traj, color=color, linewidth=1.8, alpha=0.9)
    ax.axhline(y=theta_star, color=color, linewidth=1.5, linestyle='--', alpha=0.7,
               label=f'theta* = {theta_star:.3f}')
    ax.axhline(y=0.5, color='gray', linewidth=0.8, linestyle=':', alpha=0.5,
               label='theta = 0.5 (start)')
    ax.set_xlabel('THRML round', fontsize=11)
    ax.set_ylabel('theta (spin fraction)', fontsize=11)
    ax.set_title(f'{title}\nPe(THRML) = {pe_t:+.2f}', fontsize=9.5)
    ax.set_ylim(-0.05, 1.05)
    ax.legend(fontsize=8.5)
    ax.grid(True, linestyle=':', alpha=0.3)

plt.suptitle(
    'THRML trajectories: attractive, null, repulsive void regimes\n'
    'All start from theta~0.5 (random init). Drift direction reverses sign at c_zero.',
    fontsize=10, y=1.02,
)
plt.tight_layout()
plt.savefig('nb16_repulsive_trajectories.svg', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: nb16_repulsive_trajectories.svg')

## Extended Phase Diagram

Extend the (c, K) phase diagram to c > c_zero, showing the repulsive void region.

**Key features:**
- Pe = 0 line: c = c_zero = b_alpha/b_gamma regardless of K (vertical line)
- Pe = +1 critical line (c_crit): K-dependent, converges to c_zero as K->inf
- Pe = -1 onset line: symmetric, c = (b_alpha + arcsinh(1/K)/2) / b_gamma
- As K->inf: c_crit and the Pe=-1 line both converge to c_zero
  (the diffusion zone and repulsive-onset band both vanish in the thermodynamic limit)

Religious substrates at K=16 will show the three-regime structure directly.

In [ ]:
c_vals = np.linspace(0.0, 0.65, 500)
K_vals = np.logspace(0, 5, 400)
C_mesh, K_mesh = np.meshgrid(c_vals, K_vals)

Pe_mesh = K_mesh * np.sinh(2.0 * (b_alpha - C_mesh * b_gamma))

fig, ax = plt.subplots(figsize=(12, 7))

vmax = 15
cf = ax.contourf(C_mesh, K_mesh, Pe_mesh,
                 levels=np.linspace(-vmax, vmax, 80),
                 cmap='RdBu_r', extend='both')
cbar = plt.colorbar(cf, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Pe (signed)', fontsize=11)
cbar.set_ticks([-15, -10, -5, -1, 0, 1, 5, 10, 15])

K_range = np.logspace(0, 5, 1000)

# Pe = +1 critical line
c_crit_vals = [(b_alpha - np.arcsinh(1.0 / k) / 2.0) / b_gamma for k in K_range]
valid = np.array([(0 < v < 0.65) for v in c_crit_vals])
ax.plot(np.array(c_crit_vals)[valid], K_range[valid],
        'r-', lw=2.0, label='Pe = +1  (drift onset)', zorder=5)

# Pe = -1 onset line
c_neg1_vals = [(b_alpha + np.arcsinh(1.0 / k) / 2.0) / b_gamma for k in K_range]
valid2 = np.array([(0 < v < 0.65) for v in c_neg1_vals])
ax.plot(np.array(c_neg1_vals)[valid2], K_range[valid2],
        'b-', lw=2.0, label='Pe = -1  (repulsive onset)', zorder=5)

# Pe = 0 line (c_zero, K-invariant)
ax.axvline(x=c_zero, color='white', linewidth=2.5, linestyle='--', zorder=6,
           label=f'Pe = 0  c_zero={c_zero:.4f}  (K-invariant)')

# K=16 line
ax.axhline(y=16, color='white', linewidth=1.2, linestyle=':', alpha=0.7,
           label='K = 16 (THRML default)')

# Religious substrates at K=16
religious = [
    ('JW',        float(c_from_pe_signed(-8.92)),  '#CC0066', 'P'),
    ('Mainline',  float(c_from_pe_signed(-3.23)),  '#6699CC', 's'),
    ('Buddhist',  c_zero,                           '#FF6600', 'o'),
    ('Catholic',  float(c_from_pe_signed(5.95)),   '#FFD700', 's'),
    ('Evangelical', float(c_from_pe_signed(10.55)),'#CC0000', 's'),
    ('Nones2023', float(c_from_pe_signed(18.67)),  '#888888', 'D'),
    ('Hindu',     float(c_from_pe_signed(30.0)),   '#FF9933', 'D'),
]
tech = [
    ('AI-GG', float(c_from_pe_signed(0.76)),  '#2ecc71', '^'),
    ('ETH',   float(c_from_pe_signed(3.74)),  '#627eea', 'v'),
]

for name, c_pt, color, marker in religious + tech:
    if 0.0 < c_pt < 0.65:
        ax.scatter(c_pt, 16, s=160, marker=marker, color=color,
                   edgecolors='white', linewidth=1.0, zorder=10)
        ax.text(c_pt + 0.005, 16 * 1.7, name, fontsize=7.5,
                color='white', ha='left', va='bottom')

ax.set_xlabel('Constraint level c', fontsize=13)
ax.set_ylabel('Spins per agent K', fontsize=13)
ax.set_yscale('log')
ax.set_xlim(0.0, 0.65)
ax.set_ylim(1, K_vals.max())
ax.set_title(
    'Extended phase diagram: attractive / null / repulsive void\n'
    'Red=attractive (Pe>0) | White=null (Pe=0) | Blue=repulsive (Pe<0)',
    fontsize=11,
)
ax.legend(loc='upper right', fontsize=8.5, framealpha=0.88)
plt.tight_layout()
plt.savefig('nb16_three_regime_phase.svg', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: nb16_three_regime_phase.svg')

## K-Invariance of c_zero

c_zero = b_alpha / b_gamma is a fixed point of the formula — K drops out entirely.

This is because Pe = 0 requires sinh(2*b_net) = 0, i.e., b_net = 0.
b_net = b_alpha - c*b_gamma = 0 -> c = b_alpha/b_gamma = c_zero. No K anywhere.

Contrast: c_crit satisfies Pe = 1, i.e., sinh(2*b_net) = 1/K.
As K->inf, arcsinh(1/K) -> 1/K -> 0, so c_crit -> b_alpha/b_gamma = c_zero.

The diffusion zone width (c_zero - c_crit) shrinks as K grows.
At K=16 it is only 0.014 wide. In the thermodynamic limit it vanishes.

**Consequence:** Buddhist (Pe=0, c=c_zero) sits on the K-invariant boundary.
At any hardware scale, Buddhist retention=50% corresponds to Pe=0 exactly.
The null void is a universal constant of the framework.

In [ ]:
K_test = [1, 2, 4, 8, 16, 32, 64, 128, 256, 1024, 100000]
print(f'K-invariance of c_zero vs K-dependence of c_crit:')
print(f'{"K":>8} {"c_crit":>10} {"c_zero":>10} {"gap":>12}')
print('-' * 44)
for k in K_test:
    cc = (b_alpha - np.arcsinh(1.0 / k) / 2.0) / b_gamma
    gap = c_zero - cc
    print(f'{int(k):>8} {cc:>10.6f} {c_zero:>10.6f} {gap:>+12.8f}')

print()
print(f'c_zero = {c_zero:.6f}  (constant for all K)')
print(f'c_crit -> c_zero as K -> inf (gap vanishes in thermodynamic limit)')
print()
print(f'At K->inf: no diffusion zone. Every substrate is strictly attractive or repulsive.')
print(f'The null void (Buddhist, theta*=0.5) is the only K-invariant boundary case.')

# Also: verify Buddhist theta* = 0.5 <-> Pe=0 <-> c=c_zero, for all K tested
print()
print(f'Buddhist (theta*=0.50) verification across K:')
for k in [1, 4, 16, 64, 1024]:
    pe_bud = retention_to_pe(0.5, k=k)
    c_bud = float(c_from_pe_signed(0.0))  # should always be c_zero
    print(f'  K={k:>5}: Pe(Buddhist)={pe_bud:.8f} (should be 0.0000),  c_implied={c_bud:.6f}')

## Summary

**The repulsive void exists in THRML with no new mechanism — it is c > c_zero.**

### Three regimes of Pe(c)

| c range | Pe sign | Void type | Equilibrium | Religious example |
|---------|---------|-----------|-------------|-------------------|
| c < c_crit = 0.373 | Pe > 1 | Attractive void | theta* > drift onset | Crypto, Gambling, Evangelical |
| c_crit <= c < c_zero = 0.387 | 0 < Pe < 1 | Diffusion zone | theta* in (0.5, 0.53) | AI-GG, Nones 2015 |
| c = c_zero = 0.387 | Pe = 0 | Null void | theta* = 0.50 | Buddhist (exact) |
| c > c_zero = 0.387 | Pe < 0 | Repulsive void | theta* < 0.50 | Mainline, JW |

### Key results

1. **c_zero = b_alpha/b_gamma = 0.3866 is K-invariant.**
   The null void boundary is a universal constant of the canonical parameters.
   Buddhist retention=50% <-> Pe=0 holds at any hardware scale K.

2. **JW: c_implied = 0.505 > c_zero. Pe = -8.92.**
   The 'constraint' (punitive R, shunning) has exceeded the drive (opacity + engagement).
   b_net flipped from positive to negative. The gradient runs toward EXIT.

3. **THRML trajectories confirm the sign.**
   Sampler at b_net < 0 drifts toward low-theta state (theta*=0.37).
   Sampler at b_net = 0 random-walks (no preferred direction).
   Sampler at b_net > 0 drifts toward high-theta state (ETH: theta*>0.5).

4. **Diffusion zone width = c_zero - c_crit = 0.014 at K=16.**
   Vanishes as K->inf. In the thermodynamic limit, there is no neutral zone —
   every substrate is either attractive or repulsive.

5. **The null void (Buddhist, Pe=0) is a knife-edge.**
   Not just an empirical coincidence — it is the exact condition b_net=0.
   Any perturbation (more opacity or more punitive R) tips it one way or the other.

### For 'The Congregation Effect' paper

The Pe < 0 finding is the key discriminant validity argument:
- Framework identifies REPULSIVE structures (Pe < 0), not just attractive ones
- JW and Mainline are confirmed non-voids in the engagement sense
- The void score (O/R/alpha) predicts INTENSITY; c_implied predicts DIRECTION
- These are orthogonal measurements — both needed for the full picture

**Pending:** EXP-022B (void-score scatter: O/R/alpha vs Pe for all 13 denominations)